In [ ]:
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint

In [ ]:
import zipfile,os
local_zip = '/content/hololive.zip' #change the zip filename
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [ ]:
train_dir = '/content/hololive/train' #change the middle one to the zip parent folder
validation_dir = '/content/hololive/test' #change the middle one to the zip parent folder

In [ ]:
train_datagen = ImageDataGenerator(
                    validation_split=0.2,
                    rescale=1./255)
 
test_datagen = ImageDataGenerator(
                    validation_split=0.2,
                    rescale=1./255)

In [ ]:
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    class_mode='categorical',
                                                    target_size=(150, 150),
                                                    batch_size=32,
                                                    shuffle=True)

validation_generator = train_datagen.flow_from_directory(validation_dir,
                                                         class_mode='categorical',
                                                         target_size=(150, 150),
                                                         batch_size=32,
                                                         shuffle=True)

Found 2475 images belonging to 32 classes.
Found 1600 images belonging to 32 classes.


In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(32, activation='softmax') #change the number to the number of characters in your zip file
])

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer=tf.optimizers.Adam(),
              metrics=['categorical_accuracy'])

In [ ]:
filepath = "/content/model-hololive.hdf5" #change the model name after 'model-' to the zip filename
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
desired_callbacks = [checkpoint]

In [ ]:
model.fit(
      train_generator,
      steps_per_epoch=8,
      epochs=32,
      validation_data=validation_generator,
      validation_steps=8,
      verbose=1,
      callbacks=desired_callbacks)

Epoch 1/32
8/8 [==============================] - 14s 2s/step - loss: 0.4096 - categorical_accuracy: 0.8984 - val_loss: 2.3365 - val_categorical_accuracy: 0.5508

Epoch 00001: loss did not improve from 0.27381
Epoch 2/32
8/8 [==============================] - 14s 2s/step - loss: 0.3237 - categorical_accuracy: 0.8867 - val_loss: 1.9809 - val_categorical_accuracy: 0.6016

Epoch 00002: loss did not improve from 0.27381
Epoch 3/32
8/8 [==============================] - 14s 2s/step - loss: 0.2959 - categorical_accuracy: 0.9297 - val_loss: 2.4578 - val_categorical_accuracy: 0.5195

Epoch 00003: loss did not improve from 0.27381
Epoch 4/32
8/8 [==============================] - 14s 2s/step - loss: 0.4570 - categorical_accuracy: 0.9023 - val_loss: 2.3499 - val_categorical_accuracy: 0.5391

Epoch 00004: loss did not improve from 0.27381
Epoch 5/32
8/8 [==============================] - 14s 2s/step - loss: 0.3280 - categorical_accuracy: 0.8945 - val_loss: 2.7560 - val_categorical_accuracy: 0.507